In [3]:
# [클래스가 subtop_sise_graph2인 div 태그 -> 클래스가 subtop_chart_note인 Uㅣ태그 > li태크 > 클래스가 tah인 span 태크]

import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.naver' #웹 페이지의 주소를 입력한다 
data = rq.get(url)  #request의 get()함수를 이용해 해당 페이지 내용을 받아온다. 
data_html = BeautifulSoup(data.content) #BeautifulSoup() 함수를 이용해 해당 페이지의 HTML 내용을 BeautifulSoup 객체로 만든다.
parse_day = data_html.select_one(      #select_one() 메서드를 통해 해당 태그의 데이터를 추출하며, text 메서드를 이용해 텍스트 
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text 

print(parse_day)

  |  2023.05.23


In [ ]:
# 위 과정을 통해 yyyy.mm.dd 형식의 데이터가 선택된다. 이 중 숫자 부분만 뽑아 yyyymmdd 형태로 만들어준다. 

In [4]:
import re  #regular expression (정규 표현식)

biz_day = re.findall('[0-9]+', parse_day) #findall 메서드 내에 정규 표현식을 이용해 숫자에 해당하는 부분만 추출한다. '[0-9]+ 모든 숫자를 의미하는 표현이다.'
biz_day = ''.join(biz_day) #join 함수를 통해 숫자를 합쳐준다. 

print(biz_day)

20230523


### 주식 관련 데이터를 구하기 위해 먼저 해야할 일 중 하나는 특정 증권거래소에 어떤 증권들이 상장되어 있는지 정보를 구하는 것이다. 

KRX 정보데이터시스템 http://data.krx.co.kr/ 에서 [기본통계 → 주식 → 세부안내] 부분
[12025] 업종분류 현황: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506
[12021] 개별종목: http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020502

In [5]:
# OTP(generate.cmd) / 데이터(download.cmd)
# [Headers] 탭의 'General' 항목 중 'Request URL' 부분이 원하는 항목을 제출할 주소다.
# [Payload] 탭의 Form Data에 원하는 항목이 있다. 이를 통해 POST 방식으로 데이터를 요청하면 OTP를 받음을 알 수 있다.

import requests as rq
from io import BytesIO #바이너스트림 형태 
import pandas as pd 

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd' #gen_otp_url에 항목을 제출할 URL을 입력한다. 
gen_otp_stk = {            #개발자 도구 화면에 있는 쿼리 내용들을 딕셔너리 형태로 입력한다. 
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
    }

headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020502'} #headers 부분에 referer를 입력한다. 
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text  

print(otp_stk)

3rU4AdNITLs1Tr120eSiSPMRQaH6WmZvqEBT1ac/tN0RtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eYB+B1OU8vH8x1udXwJxF+MtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [7]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd' #otp를 제출할 url을 down_url에 제출한다. 
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers = headers) #post()함수를 통해 위에서 부여받은 OTP 코드를 해당 URL에 제출한다. 
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR') # 받은 데이터의 content 부분을 ByteIO()를 이용해 바이너리스트림 형태로 만든 후, read_csv() 함수를 통해 데이터를 읽어온다.  
                                                                              # 해당 데이터는 EUC-KR 형태로 인코딩 되어 있으므로 이를 선언해준다.
sector_stk   

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4430,20,0.45,207422766850
1,006840,AK홀딩스,KOSPI,기타금융,20000,-50,-0.25,264951220000
2,027410,BGF,KOSPI,기타금융,4160,-35,-0.83,398181850560
3,282330,BGF리테일,KOSPI,유통업,183800,-1100,-0.59,3176781922800
4,138930,BNK금융지주,KOSPI,기타금융,6780,0,0.00,2209840967880
...,...,...,...,...,...,...,...,...
943,005010,휴스틸,KOSPI,철강금속,6410,150,2.40,360165560750
944,000540,흥국화재,KOSPI,보험,3290,35,1.08,211358302050
945,000547,흥국화재2우B,KOSPI,보험,17170,60,0.35,2637312000
946,000545,흥국화재우,KOSPI,보험,5530,-80,-1.43,4247040000


In [8]:
gen_otp_ksq = {            #개발자 도구 화면에 있는 쿼리 내용들을 딕셔너리 형태로 입력한다. 
    'mktId': 'KSQ',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2285,-30,-1.30,110906226970
1,054620,APS,KOSDAQ,금융,11180,-320,-2.78,228007390780
2,265520,AP시스템,KOSDAQ,반도체,21100,50,0.24,322437983100
3,211270,AP위성,KOSDAQ,통신장비,17440,-560,-3.11,263035381760
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,7590,110,1.47,310195816260
...,...,...,...,...,...,...,...,...
1634,024060,흥구석유,KOSDAQ,유통,5620,-10,-0.18,84300000000
1635,010240,흥국,KOSDAQ,기계·장비,6650,-10,-0.15,81945928400
1636,189980,흥국에프엔비,KOSDAQ,음식료·담배,3035,70,2.36,121818304945
1637,037440,희림,KOSDAQ,기타서비스,9490,-100,-1.04,132124287750


In [10]:
#코스피 데이터와 코스닥 데이터를 하나로 합친다.

krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True) #concat() 함수를 통해 두 데이터를 합쳐주며, reset_index() 메서드를 통해 인덱스를 리셋한다. drop=True를 통해 인덱스로 셋팅한 열을 삭제한다. 
krx_sector['종목명'] = krx_sector['종목명'].str.strip() #strip() 메서드를 통해 종목명에 있는 공백을 제거한다. 
krx_sector['기준일'] = biz_day #[기준일] 열 추가  

krx_sector

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4430,20,0.45,207422766850,20230523
1,006840,AK홀딩스,KOSPI,기타금융,20000,-50,-0.25,264951220000,20230523
2,027410,BGF,KOSPI,기타금융,4160,-35,-0.83,398181850560,20230523
3,282330,BGF리테일,KOSPI,유통업,183800,-1100,-0.59,3176781922800,20230523
4,138930,BNK금융지주,KOSPI,기타금융,6780,0,0.00,2209840967880,20230523
...,...,...,...,...,...,...,...,...,...
1634,024060,흥구석유,KOSDAQ,유통,5620,-10,-0.18,84300000000,20230523
1635,010240,흥국,KOSDAQ,기계·장비,6650,-10,-0.15,81945928400,20230523
1636,189980,흥국에프엔비,KOSDAQ,음식료·담배,3035,70,2.36,121818304945,20230523
1637,037440,희림,KOSDAQ,기타서비스,9490,-100,-1.04,132124287750,20230523


In [21]:
# 개별종목 데이터 크롤링 
# 앞서 살펴본 업종분류 현황 데이터 크롤링 방식과 유사, 요청하는 쿼리 값에 차이가 있음 


import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2285,-30,-1.30,33.0,69.24,NaN,NaN,829.0,2.76,0,0.00,20230523
1,095570,AJ네트웍스,4430,20,0.45,201.0,22.04,625.0,7.09,8076.0,0.55,270,6.09,20230523
2,006840,AK홀딩스,20000,-50,-0.25,NaN,NaN,NaN,NaN,41948.0,0.48,200,1.00,20230523
3,054620,APS,11180,-320,-2.78,505.0,22.14,NaN,NaN,10864.0,1.03,0,0.00,20230523
4,265520,AP시스템,21100,50,0.24,5463.0,3.86,3534.0,5.97,17980.0,1.17,270,1.28,20230523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532,000547,흥국화재2우B,17170,60,0.35,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,20230523
2533,000545,흥국화재우,5530,-80,-1.43,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,20230523
2534,003280,흥아해운,1385,-17,-1.21,94.0,14.73,NaN,NaN,544.0,2.55,0,0.00,20230523
2535,037440,희림,9490,-100,-1.04,567.0,16.74,NaN,NaN,5186.0,1.83,150,1.58,20230523


In [12]:
#데이터 정리하기 
# 두 데이터에 공통으로 존재하지 않는 종목,하나의 데이터에만 존재하는 항목을 구별

diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['이지스밸류리츠', '케이탑리츠', '마스턴프리미어리츠', '윙입푸드', '코오롱티슈진', '헝셩그룹', '로스웰', '에이리츠', '미래에셋글로벌리츠', '잉글우드랩', 'SBI핀테크솔루션즈', '프레스티지바이오파마', '글로벌에스엠', '코람코더원리츠', '엑세스바이오', '한국ANKOR유전', '한국패러랠', '코람코에너지리츠', 'GRT', '신한알파리츠', '이리츠코크렙', '미투젠', 'ESR켄달스퀘어리츠', 'SK리츠', '디앤디플랫폼리츠', '모두투어리츠', '크리스탈신소재', '엘브이엠씨홀딩스', '신한서부티엔디리츠', 'NH프라임리츠', '이지스레지던스리츠', '맥쿼리인프라', '미래에셋맵스리츠', '이스트아시아홀딩스', '네오이뮨텍', '컬러레이', '소마젠', '제이알글로벌리츠', '애머릿지', '삼성FN리츠', '씨케이에이치', '롯데리츠', '골든센츄리', '맵스리얼티1', 'JTC', '바다로19호', '한화리츠', '오가닉티코스메틱', 'NH올원리츠', 'KB스타리츠']


In [22]:
kor_ticker = pd.merge(krx_sector,     #merge() 함수는 on 조건을 기준으로 데이터를 합치며, intersection() 메서드르 이용해 공통으로 존재하는 열을 기준으로 입력해 준다. 
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')  # join 방식: outer 합집합 

kor_ticker

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4430,20,0.45,207422766850,20230523,201.0,22.04,625.0,7.09,8076.0,0.55,270.0,6.09
1,006840,AK홀딩스,KOSPI,기타금융,20000,-50,-0.25,264951220000,20230523,NaN,NaN,NaN,NaN,41948.0,0.48,200.0,1.00
2,027410,BGF,KOSPI,기타금융,4160,-35,-0.83,398181850560,20230523,247.0,16.84,NaN,NaN,16528.0,0.25,110.0,2.64
3,282330,BGF리테일,KOSPI,유통업,183800,-1100,-0.59,3176781922800,20230523,11203.0,16.41,13632.0,13.48,55724.0,3.30,4100.0,2.23
4,138930,BNK금융지주,KOSPI,기타금융,6780,0,0.00,2209840967880,20230523,2404.0,2.82,2603.0,2.60,30468.0,0.22,625.0,9.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2582,024060,흥구석유,KOSDAQ,유통,5620,-10,-0.18,84300000000,20230523,183.0,30.71,NaN,NaN,5508.0,1.02,150.0,2.67
2583,010240,흥국,KOSDAQ,기계·장비,6650,-10,-0.15,81945928400,20230523,740.0,8.99,NaN,NaN,7971.0,0.83,220.0,3.31
2584,189980,흥국에프엔비,KOSDAQ,음식료·담배,3035,70,2.36,121818304945,20230523,309.0,9.82,298.0,10.17,2295.0,1.32,40.0,1.32
2585,037440,희림,KOSDAQ,기타서비스,9490,-100,-1.04,132124287750,20230523,567.0,16.74,NaN,NaN,5186.0,1.83,150.0,1.58


In [14]:
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

['엔에이치스팩19호' 'DB금융스팩10호' 'DB금융스팩8호' 'DB금융스팩9호' 'IBKS제17호스팩' 'IBKS제19호스팩'
 'IBKS제20호스팩' 'IBKS제21호스팩' 'IBKS제22호스팩' '교보11호스팩' '교보12호스팩' '교보13호스팩'
 '대신밸런스제13호스팩' '대신밸런스제14호스팩' '미래에셋드림스팩1호' '미래에셋비전스팩1호' '미래에셋비전스팩2호'
 '미래에셋비전스팩3호' '비엔케이제1호스팩' '삼성머스트스팩5호' '삼성스팩4호' '삼성스팩6호' '삼성스팩7호' '삼성스팩8호'
 '상상인제3호스팩' '신영스팩7호' '신영스팩8호' '신영스팩9호' '신한제10호스팩' '신한제7호스팩' '신한제8호스팩'
 '신한제9호스팩' '에스케이증권제8호스팩' '에이치엠씨제5호스팩' '엔에이치스팩20호' '엔에이치스팩23호' '엔에이치스팩24호'
 '엔에이치스팩25호' '엔에이치스팩26호' '엔에이치스팩27호' '엔에이치스팩28호' '유안타제10호스팩' '유안타제12호스팩'
 '유안타제13호스팩' '유안타제8호스팩' '유안타제9호스팩' '유진스팩6호' '유진스팩7호' '유진스팩8호' '유진스팩9호'
 '케이비제21호스팩' '케이비제22호스팩' '케이비제23호스팩' '키움제6호스팩' '키움제7호스팩' '키움제8호스팩'
 '하나26호스팩' '하나27호스팩' '하나금융20호스팩' '하나금융21호스팩' '하나금융22호스팩' '하나금융23호스팩'
 '하나금융24호스팩' '하나금융25호스팩' '하나머스트7호스팩' '하이제6호스팩' '하이제7호스팩' '한국제10호스팩'
 '한국제11호스팩' '한화플러스제2호스팩' '한화플러스제3호스팩']


In [15]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values) # 국내 주식에서 끝이 0이 아닌 종목은 우선주에 해당함 

['BYC우' 'CJ4우(전환)' 'CJ씨푸드1우' 'CJ우' 'CJ제일제당 우' 'DB하이텍1우' 'DL우'
 'DL이앤씨2우(전환)' 'DL이앤씨우' 'GS우' 'JW중외제약2우B' 'JW중외제약우' 'LG생활건강우' 'LG우'
 'LG전자우' 'LG화학우' 'LX하우시스우' 'LX홀딩스1우' 'NH투자증권우' 'NPC우' 'S-Oil우' 'SK네트웍스우'
 'SK디스커버리우' 'SK우' 'SK이노베이션우' 'SK증권우' 'SK케미칼우' '계양전기우' '금강공업우' '금호건설우'
 '금호석유우' '깨끗한나라우' '남선알미우' '남양유업우' '넥센우' '넥센타이어1우B' '노루페인트우' '노루홀딩스우'
 '녹십자홀딩스2우' '대교우B' '대덕1우' '대덕전자1우' '대상우' '대상홀딩스우' '대신증권2우B' '대신증권우'
 '대원전선우' '대한제당우' '대한항공우' '덕성우' '동부건설우' '동양2우B' '동양우' '동원시스템즈우' '두산2우B'
 '두산우' '두산퓨얼셀1우' '두산퓨얼셀2우B' '롯데지주우' '롯데칠성우' '미래에셋증권2우B' '미래에셋증권우' '부국증권우'
 '삼성SDI우' '삼성물산우B' '삼성전기우' '삼성전자우' '삼성중공우' '삼성화재우' '삼양사우' '삼양홀딩스우' '서울식품우'
 '성문전자우' '성신양회우' '세방우' '솔루스첨단소재1우' '솔루스첨단소재2우B' '신영증권우' '신풍제약우'
 '아모레G3우(전환)' '아모레G우' '아모레퍼시픽우' '유안타증권우' '유유제약1우' '유유제약2우B' '유한양행우'
 '유화증권우' '일양약품우' '진흥기업2우B' '진흥기업우B' '코리아써우' '코리아써키트2우B' '코오롱글로벌우'
 '코오롱모빌리티그룹우' '코오롱우' '코오롱인더우' '크라운제과우' '크라운해태홀딩스우' '태양금속우' '태영건설우'
 '티와이홀딩스우' '하이트진로2우B' '하이트진로홀딩스우' '한국금융지주우' '한양증권우' '한진칼우' '한화3우B'
 '한화갤러리아우' '한화솔루션우' '한화우' '한화투자증권우' '현

In [16]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values) #이리츠코크랩이 누락되어 있음, 데이터베이스에서 SQL로 변경하는 전처리 필요 

['ESR켄달스퀘어리츠' 'KB스타리츠' 'NH올원리츠' 'NH프라임리츠' 'SK리츠' '디앤디플랫폼리츠' '롯데리츠'
 '마스턴프리미어리츠' '모두투어리츠' '미래에셋글로벌리츠' '미래에셋맵스리츠' '삼성FN리츠' '신한서부티엔디리츠' '신한알파리츠'
 '에이리츠' '이지스레지던스리츠' '이지스밸류리츠' '제이알글로벌리츠' '케이탑리츠' '코람코더원리츠' '코람코에너지리츠'
 '한화리츠']


In [24]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',    #numpy 패키지의 where() 함수를 통해 각 조건에 맞는 종목구분을 입력한다. 
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)  #reset_index() 메서드를 통해 인덱스를 초기화한다. 
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '') #replace() 메서드를 통해 열 이름의 공백을 삭제한다. 
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']] #필요한 열만 선택한다.
kor_ticker = kor_ticker.replace({np.nan: None})  #SQL에서는 NaN이 입력되지 않음, 삭제 필요 
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일']) #to_datetime() 메서드를 이용해 yyyymmddd에서 yyyy-mm-dd 형태로 변경

kor_ticker

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4430,207422766850,2023-05-23,201.0,625.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,20000,264951220000,2023-05-23,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,4160,398181850560,2023-05-23,247.0,None,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,183800,3176781922800,2023-05-23,11203.0,13632.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,6780,2209840967880,2023-05-23,2404.0,2603.0,30468.0,625.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2582,024060,흥구석유,KOSDAQ,5620,84300000000,2023-05-23,183.0,None,5508.0,150.0,보통주
2583,010240,흥국,KOSDAQ,6650,81945928400,2023-05-23,740.0,None,7971.0,220.0,보통주
2584,189980,흥국에프엔비,KOSDAQ,3035,121818304945,2023-05-23,309.0,298.0,2295.0,40.0,보통주
2585,037440,희림,KOSDAQ,9490,132124287750,2023-05-23,567.0,None,5186.0,150.0,보통주


create database stock_db;

use stock_db;

create table kor_ticker
(
    종목코드 varchar(6) not null,
    종목명 varchar(20),
    시장구분 varchar(6),
    종가 float,
    시가총액 float,
    기준일 date,
    EPS float,
    선행EPS float,
    BPS float,
    주당배당금 float,
    종목구분 varchar(5),
    primary key(종목코드, 기준일)
  );

In [19]:
# 파이썬에서 아래 코드를 실행하면 다운로드 받은 정보가 kor_ticker 테이블에 upsert 형태로 저장된다. 

import pymysql

con = pymysql.connect(user='root', #사용자 명 
                      passwd='1234', # 비밀번호
                      host='127.0.0.1', # 허용 접속 IP
                      db='stock_db', # 사용할 데이터베이스
                      charset='utf8') # 인코딩 방법 

mycursor = con.cursor()   #커서 객체를 불러옴 
query = f"""          
    insert into kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
    values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    종목명=new.종목명,시장구분=new.시장구분,종가=new.종가,시가총액=new.시가총액,EPS=new.EPS,선행EPS=new.선행EPS,
    BPS=new.BPS,주당배당금=new.주당배당금,종목구분 = new.종목구분;
"""
###upsert 형태로 입력 (덮어쓰여지는게 아니라 더해집)
### on duplicate key update; primary key에 해당하는 종목코드/기준일을 제외한 모든 열을 입력 

args = kor_ticker.values.tolist()  #values를 list 형태로 바꿈 

mycursor.executemany(query, args)
con.commit() #데이터 전송 완료 

con.close() # DB 서버와 연결 종료